# Intelligent Query Processing
## Approximate Count Distinct 
Approximate Count Distinct is an example of Intelligent Query Processing that can be enabled just by upgrading to SQL Server 2019 (dbcompat of 150 is not required). Follow the steps in this notebook to see the performance difference when using the new **APPROX_COUNT_DISTINCT** T-SQL function vs using the combination of **COUNT** and **DISTINCT**.

**Prerequisites**: Restore the WideWorldImportersDW full backup. Then run the **extendwwidw.sql** script to customize the database. If you have already done this, proceed to the next steps

## Step 1: Do some setup
Clear the procedure cache to ensure no other previous runs are affected, set dbcompat to 130 just to prove you don't need 150, and warm the buffer pool cache with pages from OrderHistoryExtended to make the comparison fair.

In [1]:
-- Step 1: Clear the cache, set dbcompat to 130 jsut to prove it works, and warm the cache
USE WideWorldImportersDW
GO
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE
GO
ALTER DATABASE WideWorldImportersDW SET COMPATIBILITY_LEVEL = 130
GO
SELECT COUNT(*) FROM Fact.OrderHistoryExtended
GO

Commands completed successfully.

Total execution time: 00:00:00.0010290

Commands completed successfully.

Total execution time: 00:00:00.0040395

Commands completed successfully.

Total execution time: 00:00:00.0647228

(1 row affected)

Total execution time: 00:00:02.9276605

(No column name)
29620736


## Step 2: Use COUNT and DISTINCT first
Find the number of distinct order ID values in the OrderHistoryExtended table. This table has somwhere around 29 million rows. One execution of this query may take a few seconds but what if we run it 10 times?

In [2]:
-- Step 2: Use COUNT and DISTINCT first. Do it 10 times to help see the performance difference
SELECT COUNT(DISTINCT [WWI Order ID])
FROM [Fact].[OrderHistoryExtended]
GO

(1 row affected)

Total execution time: 00:00:06.5499056

(No column name)
29620736


## Step 3: Now run the same query but use the new APPROX_COUNT_DISTINCT T-SQL function.
This function only requires SQL Server 2019 or higher. SQL Server has a 2% margin of error on the actual distinct count at a probabilty of 97%. For this table, that is +- 592,414 rows. You should see about a 2x+ faster performance from the same query 10 times

In [3]:
-- Step 5: Use the new APPROX_COUNT_DISTINCT function to compare values and performance
-- We should be no more than 2% off the actual distinct value (97% probability)
SELECT APPROX_COUNT_DISTINCT([WWI Order ID])
FROM [Fact].[OrderHistoryExtended]
GO

(1 row affected)

Total execution time: 00:00:02.1383592

(No column name)
29852585


## Step 4: Restore dbcompat

In [0]:
-- Step 4: Restore database compatibility level
ALTER DATABASE WideWorldImportersDW SET COMPATIBILITY_LEVEL = 150
GO

Commands completed successfully.

Total execution time: 00:00:00.0091388